In [1]:
import pandas as pd
import numpy as np
import datetime as dt

from util_funcs import *

### Loading data

In [2]:
PATH_TO_INPUT_FILES = './data/'

In [17]:
df_pop = pd.read_csv(PATH_TO_INPUT_FILES+'italy_reg_pop_DEF.csv')
df_covid = pd.read_csv(PATH_TO_INPUT_FILES+'italy_cases_recovered_deaths_DEF.csv', parse_dates=['date'])
df_flows = pd.read_csv(PATH_TO_INPUT_FILES+'italy_movement_24_feb_22_apr_DEF.csv', parse_dates=['date'])
df_lockdown = pd.read_csv(PATH_TO_INPUT_FILES+'italy_lockdown_data.csv', parse_dates=['date'])

In [4]:
df_covid.tail()

,date,reg_name,positives,delta_positives,removed,delta_removed
1275,2020-04-27,TOSCANA,5983,-86,3196,118
1276,2020-04-27,TRENTINO ALTO ADIGE,2647,-29,3844,145
1277,2020-04-27,UMBRIA,287,-9,1083,11
1278,2020-04-27,VALLE D'AOSTA,235,-19,876,24
1279,2020-04-27,VENETO,8860,-278,8719,386


### The model


$\Delta S_r(t) = - \beta_r \dfrac{I_r(t) S_r(t)}{N_r(t)}$

$\Delta I_r(t) = \beta_r \bigg{(}I_r(t) + \sum_{r' \ne r} p_{r'} L_{r',r}(t) \dfrac{I_{r'}(t)}{N_{r'}(t)} \bigg{)} \dfrac{S_r(t)}{N_r(t)} - \gamma_r \dfrac{I_r(t)}{N_r(t)}$

$\Delta R_r(t) = \gamma_r \dfrac{I_r(t)}{N_r(t)}$

where $L_{r',r}(t)$ is the total movement inflow to region $r$ from all the other regions at time $t$, and $p_r$ is the probability that someone who is infected will travel.

#### Compute $S_r(t)$, $\Delta S_r(t)$

Loaded datasets give us:

$I_r(t)$, $\Delta I_r(t)$, $R_r(t)$, $\Delta R_r(t)$ $\rightarrow$ respectively `positives`, `delta_positives`, `removed`, `delta_removed` in DataFrame `df_covid`

$N_r(t) = N_r$ $\rightarrow$ `pop` in DataFrame `df_pop`

Then, we need to derive:

$S_r(t)$, $\Delta S_r(t)$

In [5]:
def add_pop_to_df(df_covid, df_pop):
    
    df_with_N = df_covid.groupby(['reg_name']).apply(_add_N, df_pop)
    df_with_N = df_with_N[['date', 'reg_name', 'population'] + list(df_covid.columns[2:])]
    
    return df_with_N

def _add_N(df, df_pop):
    
    population = int(df_pop[df_pop['reg_name'] == df['reg_name'].iloc[0]]['pop'])
    array_N = np.repeat(population, df.shape[0]) # N_r is fixed, it does not change with t
    df['population'] = array_N
    
    return df

def add_susceptibles_to_df(df_covid, df_pop):
    
    if 'population' not in df_covid.columns:
        df_covid = add_pop_to_df(df_covid, df_pop).copy()
    
    df_with_S = df_covid.groupby(['reg_name']).apply(_add_S)
    df_with_S = df_with_S[['date', 'reg_name', 'population', 'susceptibles', 'delta_susceptibles'] + list(df_covid.columns[3:])]
    
    return df_with_S

def _add_S(df):
    
    df['susceptibles'] = df['population'] - (df['positives'] + df['removed'])
    
    array_S = np.array(df['susceptibles'])
    array_delta_S = np.insert(np.diff(array_S), 0, array_S[0])
    df['delta_susceptibles'] = array_delta_S
    
    return df

In [6]:
df_all = add_susceptibles_to_df(df_covid, df_pop)

# showing the result for one region...
df_all[df_all['reg_name'] == 'PIEMONTE'].head(10)

,date,reg_name,population,susceptibles,delta_susceptibles,positives,delta_positives,removed,delta_removed
11,2020-02-24,PIEMONTE,4356407,4356404,4356404,3,3,0,0
31,2020-02-25,PIEMONTE,4356407,4356404,0,3,0,0,0
51,2020-02-26,PIEMONTE,4356407,4356404,0,3,0,0,0
71,2020-02-27,PIEMONTE,4356407,4356405,1,2,-1,0,0
91,2020-02-28,PIEMONTE,4356407,4356396,-9,11,9,0,0
111,2020-02-29,PIEMONTE,4356407,4356396,0,11,0,0,0
131,2020-03-01,PIEMONTE,4356407,4356358,-38,49,38,0,0
151,2020-03-02,PIEMONTE,4356407,4356356,-2,51,2,0,0
171,2020-03-03,PIEMONTE,4356407,4356351,-5,56,5,0,0
191,2020-03-04,PIEMONTE,4356407,4356325,-26,82,26,0,0


####  Data aggregation

Covid-19 data are stored at a daily basis.

In our model, we will use aggregated data, then we need an aggregation...

In [7]:
def aggregate_SIR_data(df_covid, num_of_days_per_group = 7):
    
    df_aggr = df_covid.groupby(['reg_name']).apply(_aggregate, num_of_days_per_group).reset_index()
    df_aggr = df_aggr[list(df_covid.columns)]
    
    return df_aggr

def _aggregate(df, num_of_days_per_group):

    # creating the labels for the aggregation
    # these are stored in a temporary new column ('group_labels') of type [0,0,0,1,1,1,2,2,2,...,k,k,k] if the num_of_days_per_group = 3
    num_of_entries = df.shape[0]
    days = np.arange(0,num_of_entries)
    group_labels = np.repeat(days, num_of_days_per_group)[0:num_of_entries]
    df['group_labels'] = group_labels

    # aggregating the deltas (delta_positives and delta_removed)
    df_aggr = df.groupby(['group_labels']).agg({'delta_susceptibles':'sum', 'delta_positives':'sum', 'delta_removed':'sum'}).reset_index()

    # for population, positives and removed, the value of the last day in each aggregated group is the value we want for that group:
    last_indexes = [group_indexes[len(group_indexes)-1] for group_indexes in df.groupby(['group_labels']).groups.values()]
    df_aggr['susceptibles'] = np.array(df.loc[last_indexes, 'susceptibles'])
    df_aggr['positives'] = np.array(df.loc[last_indexes, 'positives'])
    df_aggr['removed'] = np.array(df.loc[last_indexes, 'removed'])
    df_aggr['population'] = np.array(df.loc[last_indexes, 'population'])

    # the date of each aggregated group will be the date of the first day in each group:
    first_indexes = [group_indexes[0] for group_indexes in df.groupby(['group_labels']).groups.values()]
    df_aggr['date'] = np.array(df.loc[first_indexes, 'date'])

    return df_aggr

If we have $N$ days, as we have seen above, the first $k$ of them will be organized like this:

| date  | $N_r$      | $S_r$      | $\Delta S_r$      | $I_r$      | $\Delta I_r$      | $R_r$      | $\Delta R_r$      |
|-------|------------|------------|-------------------|------------|-------------------|------------|-------------------|
| $d_0$ | $N_r(d_0)$ | $S_r(d_0)$ | $\Delta S_r(d_0)$ | $I_r(d_0)$ | $\Delta I_r(d_0)$ | $R_r(d_0)$ | $\Delta R_r(d_0)$ |
| $d_1$ | $N_r(d_1)$ | $S_r(d_1)$ | $\Delta S_r(d_1)$ | $I_r(d_1)$ | $\Delta I_r(d_1)$ | $R_r(d_1)$ | $\Delta R_r(d_1)$ |
|  ...  |            |            |                   |            |                   |            |                   |
| $d_k$ | $N_r(d_k)$ | $S_r(d_k)$ | $\Delta S_r(d_k)$ | $I_r(d_k)$ | $\Delta I_r(d_k)$ | $R_r(d_k)$ | $\Delta R_r(d_k)$ |

If we aggregate the data into $\big{\lceil}\frac{N}{k}\big{\rceil}$ groups of $k$ days, the $k$ rows of the table above will be grouped in the first group, say $D = (d_0, d_1, ..., d_k)$, thus resulting in one row in te aggregated DataFrame:

| date  | $N_r$      | $S_r$                      | $\Delta S_r$    | $I_r$                      | $\Delta I_r$    | $R_r$                      | $\Delta R_r$    |
|-------|------------|----------------------------|-----------------|----------------------------|-----------------|----------------------------|-----------------|
| $d_0$ | $N_r(d_k)$ | $S_r(d_0) + \Delta S_r(D)$ | $\Delta S_r(D)$ | $I_r(d_0) + \Delta I_r(D)$ | $\Delta I_r(D)$ | $R_r(d_0) + \Delta R_r(D)$ | $\Delta R_r(D)$ |

where 

$\Delta S_r(D) = \sum_{i=0}^k \Delta S_r(d_i)$

$\Delta I_r(D) = \sum_{i=0}^k \Delta I_r(d_i)$

$\Delta R_r(D) = \sum_{i=0}^k \Delta R_r(d_i)$

The `date` indicating the group will just be the date of the first day in the group (in the example, $d_0$).

As in our model $N_r(t) = N_r$, the value of population of region $r$ is simply its value in the last day in the group (in the example, $N_r(d_k)$).

In [8]:
df_aggregated = aggregate_SIR_data(df_all, 7)

df_aggregated[df_aggregated['reg_name'] == 'PIEMONTE'].head(10)

,date,reg_name,population,susceptibles,delta_susceptibles,positives,delta_positives,removed,delta_removed
110,2020-02-24,PIEMONTE,4356407,4356358,4356358,49,49,0,0
111,2020-03-02,PIEMONTE,4356407,4356047,-311,355,306,5,5
112,2020-03-09,PIEMONTE,4356407,4355296,-751,1030,675,81,76
113,2020-03-16,PIEMONTE,4356407,4351987,-3309,4127,3097,293,212
114,2020-03-23,PIEMONTE,4356407,4348201,-3786,7268,3141,938,645
115,2020-03-30,PIEMONTE,4356407,4344045,-4156,10177,2909,2185,1247
116,2020-04-06,PIEMONTE,4356407,4339747,-4298,12505,2328,4155,1970
117,2020-04-13,PIEMONTE,4356407,4335350,-4397,14470,1965,6587,2432
118,2020-04-20,PIEMONTE,4356407,4331587,-3763,15519,1049,9301,2714
119,2020-04-27,PIEMONTE,4356407,4331309,-278,15508,-11,9590,289


#### Movement flows

In the equations, we need $L_{r',r}(t)$, i.e. the movement flows from $r'$ to $r$ during $t$.

These are stored in the DataFrame `df_flows`, with two levels of spatial aggregation: `provincia` and `regione`.

In [9]:
df_flows.head()

,date,start_prov_name,end_prov_name,flow,start_reg_name,end_reg_name
0,2020-02-24,LIVORNO,PISTOIA,12,TOSCANA,TOSCANA
1,2020-02-24,PARMA,REGGIO NELL'EMILIA,691,EMILIA ROMAGNA,EMILIA ROMAGNA
2,2020-02-24,SALERNO,POTENZA,49,CAMPANIA,BASILICATA
3,2020-02-24,PARMA,MANTOVA,67,EMILIA ROMAGNA,LOMBARDIA
4,2020-02-24,FOGGIA,AVELLINO,13,PUGLIA,CAMPANIA


A few functions are useful to extract movement flows between different regions in arbitrary periods.

For example, lets extract the flow from Toscana to Lazio between the 24th and the 28th of February, i.e. at the very beginning of the crisis:

In [10]:
extract_flow_in_time_interval(df_flows, 'TOSCANA', 'LAZIO',
                              t_0 = dt.datetime(2020, 2, 24, 0, 0, 0), t_1 = dt.datetime(2020, 2, 28, 0, 0, 0),
                              type_of_region = 'regione')

933

The following week (between the 2nd and the 6th of March), the flow decreased by almost 50% :

In [11]:
extract_flow_in_time_interval(df_flows, 'TOSCANA', 'LAZIO',
                              t_0 = dt.datetime(2020, 3, 2, 0, 0, 0), t_1 = dt.datetime(2020, 3, 6, 0, 0, 0),
                              type_of_region = 'regione')

491

And during the second week of March (9th-13th, when the lockdown started), it is decreased by 68% :

In [12]:
extract_flow_in_time_interval(df_flows, 'TOSCANA', 'LAZIO',
                              t_0 = dt.datetime(2020, 3, 9, 0, 0, 0), t_1 = dt.datetime(2020, 3, 13, 0, 0, 0),
                              type_of_region = 'regione')

298

We can also extract in/out-flows from/to a certain region to/from all the other regions (i.e. our $L_{r',r}(t)$ in the model).

E.g. for Lombardia in the second week of March, we have:

In [13]:
inflow = extract_inflow_in_time_interval(df_flows, 'LOMBARDIA', 
                                         t_0 = dt.datetime(2020, 3, 9, 0, 0, 0), t_1 = dt.datetime(2020, 3, 13, 0, 0, 0), 
                                         type_of_region = 'regione')
outflow = extract_outflow_in_time_interval(df_flows, 'LOMBARDIA', 
                                           t_0 = dt.datetime(2020, 3, 9, 0, 0, 0), t_1 = dt.datetime(2020, 3, 13, 0, 0, 0), 
                                           type_of_region = 'regione')
print('In-flow: ', inflow)
print('Out-flow: ', outflow)

In-flow:  10642
Out-flow:  8991


#### A possible model for $\beta_r$ and $p_r$

The [Oxford COVID-19 Government Response Tracker](https://covidtracker.bsg.ox.ac.uk/) collects information on common policy responses, scores the stringency of such measures, and aggregates these into a Stringency Index. 

The Stringency Index is based on 9 indicators; 6 of them could be useful to describe our $\beta_r$: school closing ($C_1$), workplace closing ($C_2$), cancel public events ($C_3$), restrictions on gatherings ($C_4$), close public transport ($C_5$), stay at home requirements ($C_6$). While one of them, restrictions on internal movement ($C_7$), could somehow be our $p_r$.

As the indicators are computed at a country-level, we will have $\beta_r = \beta$ and $p_r = p$.

Each indicator $I_j$ takes values in an ordinal scale from 0 to $N_j$, and it also has a flag $G_j \in \{0,1\}$ that indicates whether the policy is generally applied (i.e. to the whole country) or not (i.e. it is applied only on target regions).

Every of them is computed as:

$I_j = C_j \dfrac{1-w}{N_j} + w G_j, \quad j \in \{1,2,...,7\}$

where $w = \frac{1}{7} \sum_{j = 1}^7 \frac{1}{N_j+1}$.

Each indicator takes a standardized ‘bonus point’ for a generally-applied policy, and we have $0 \leq I_j \leq 1$. 

Then, our $\beta$ could be modeled as:

$\beta = \sum_{j = 1}^6 \alpha_j^{\beta} I_j + c^{\beta}$.



In [22]:
df_lockdown.tail()

,date,C1,C1_Flag,C2,C2_Flag,C3,C3_Flag,C4,C4_Flag,C5,C5_Flag,C6,C6_Flag,C7,C7_Flag,C8,stringency_index
116,2020-04-26,3.0,1.0,3.0,1.0,NaN,NaN,NaN,NaN,2.0,1.0,NaN,NaN,2.0,1.0,2.0,NaN
117,2020-04-27,3.0,1.0,3.0,1.0,NaN,NaN,NaN,NaN,2.0,1.0,NaN,NaN,2.0,1.0,2.0,NaN
118,2020-04-28,3.0,1.0,3.0,1.0,NaN,NaN,NaN,NaN,2.0,1.0,NaN,NaN,2.0,1.0,2.0,NaN
119,2020-04-29,3.0,1.0,3.0,1.0,NaN,NaN,NaN,NaN,2.0,1.0,NaN,NaN,2.0,1.0,2.0,NaN
120,2020-04-30,3.0,1.0,3.0,1.0,NaN,NaN,NaN,NaN,2.0,1.0,NaN,NaN,2.0,1.0,2.0,NaN


In [20]:
map_Indicator_to_N = {
    '1': 3,
    '2': 3,
    '3': 2,
    '4': 4,
    '5': 2,
    '6': 3,
    '7': 2,
    '8': 4
}

w = (1/8) * sum([1/(N+1) for N in map_Indicator_to_N.values()])

def compute_I(C, C_num, G=0):
    if C_num != '8':
        I = C * (1-w) / map_Indicator_to_N[C_num] + w * G
    else:
        # this is for indicator 8, that has no flag 1/0
        I = C / map_Indicator_to_N[C_num]
    return I

def add_I_to_df(df_lockdown):
    
    df_with_I = df_lockdown.copy()
    
    for C_num in map_Indicator_to_N.keys():
        if C_num != '8':
            df_with_I['I_'+C_num] = compute_I(df_with_I['C'+C_num], C_num, df_with_I['C'+C_num+'_Flag'])
        else:
            df_with_I['I_'+C_num] = compute_I(df_with_I['C'+C_num], C_num)
        
    return df_with_I

df_Ind = add_I_to_df(df_lockdown)[['date', 'I_1', 'I_2', 'I_3', 'I_4', 'I_5', 'I_6', 'I_7', 'I_8']]
df_Ind.tail()

,date,I_1,I_2,I_3,I_4,I_5,I_6,I_7,I_8
116,2020-04-26,1.0,1.0,NaN,NaN,1.0,NaN,1.0,0.5
117,2020-04-27,1.0,1.0,NaN,NaN,1.0,NaN,1.0,0.5
118,2020-04-28,1.0,1.0,NaN,NaN,1.0,NaN,1.0,0.5
119,2020-04-29,1.0,1.0,NaN,NaN,1.0,NaN,1.0,0.5
120,2020-04-30,1.0,1.0,NaN,NaN,1.0,NaN,1.0,0.5


#### Writing the model

In [ ]:
from scipy.integrate import odeint
from sklearn.linear_model import LinearRegression, LogisticRegression

def deriv(y, t, beta, gamma, p, o, N, name_of_region):
    
    S, I, R = y
    
    def compute_I_out(name_of_region):
    
    I_out = 0
    for region in list_all_regions:
        if region != name_of_region:
            L = ### TODO
            I = 
            N = 
            I_out += p(t) * L * I / N
    
    return I_out

    ##
    dSdt = -beta(t) * I * S / N

    I_out = compute_I_out(name_of_region)
    dIdt = beta(t) * ((I + I_out) * S) / N - gamma * I / N
    
    dRdt = gamma * I / N
    ##
    
    return dSdt, dIdt, dRdt

##
gamma = 1.0/3.0
list_all_regions = list(set(df_covid['reg_name']))
##

def model(days, df_covid_data, name_of_region):
    
    df_covid_region = df_covid_data[df_covid_data['reg_name'] == name_of_region]

    N = int(df_covid_region['population'].iloc[0]) # pop fixed
    
    ###
    def beta(t):
        Ind = np.array(df_Ind[['I_1', 'I_2', 'I_3', 'I_4', 'I_5', 'I_6']].loc[t])
        linreg = LinearRegression().fit(Ind, beta_start)
        alpha = linreg.coef_
        c = linreg.intercept_
        return alpha, c
    
    def p(t):
        Ind = np.array(df_Ind['I_7'].loc[t])
        logreg = LogisticRegression().fit(Ind, p_start)
        alpha = logreg.coef_
        c = logreg.intercept_
        return alpha, c
    
    def o(t):
        Ind = np.array(df_Ind['I_8'].loc[t])
        linreg = LinearRegression().fit(Ind, o_start)
        alpha = linreg.coef_
        c = linreg.intercept_
        return alpha, c
    ###
    
    y0 = N-1.0, 1.0, 0.0 # S, I, R
    t = np.linspace(0, days-1, days)
    ret = odeint(deriv, y0, t, args=(beta, gamma, p, o, N, name_of_region))
    S, I, R = ret.T
    
    ###
    beta_t = #### TODO
    ###

    return t, S, I, R

#### Fitting the data

In [ ]:
for c_name_of_region in list_all_regions:
    
    df_covid_region = df_covid[df_covid['reg_name'] == c_name_of_region]
    
    days = len(df_covid_region['date'])
    
    np.linspace(0, days-1, days, dtype=int)
    
    #### TODO
    